%load_ext autoreload
%autoreload 2
https://blog.csdn.net/m0_46413065/article/details/129917287

In [195]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [196]:
data_g2 = pd.read_csv("./Data/Kaohsiung/g2.csv",delimiter=';')
data_g3 = pd.read_csv("./Data/Kaohsiung/g3.csv",delimiter=';')
data_g4 = pd.read_csv("./Data/Kaohsiung/g4.csv",delimiter=';')
data = [data_g2]

features = ['date_time']
targets = ['TotalMove']
group_id_i = 2

for i in range(1):
    data[i]['date_time'] = pd.to_datetime(data[i]['date_time'])
    data[i]['date_time'] = data[i]['date_time'].astype(int) / (10 ** 11)
    data[i]['date_time'] = data[i]['date_time'].astype(int)
    data[i] = data[i][features+targets]
    #data[i]['group_id'] = group_id_i
    group_id_i += 1
    
data = pd.concat(data,ignore_index=True)
data['group_id'] = 0

In [197]:
data.head()

,date_time,TotalMove,group_id
0,17055522,0.0,0
1,17055528,1.6,0
2,17055744,18.7,0
3,17055750,31.1,0
4,17055762,21.5,0


In [198]:
data.tail()

,date_time,TotalMove,group_id
31179,17355504,537.6,0
31180,17355510,537.6,0
31181,17355516,537.8,0
31182,17355522,532.5,0
31183,17355600,534.5,0


In [199]:
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import MultiNormalizer,TorchNormalizer

max_prediction_length = 7*144
max_encoder_length = 30*144

training = TimeSeriesDataSet(
    data[:len(data)-max_prediction_length],
    group_ids = ["group_id"],
    target = 'TotalMove',
    time_idx = "date_time",
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_encoder_length=max_encoder_length//2,  # 似乎要自己設置(默認值好像太大?)
    min_prediction_length=1,  #同上
    time_varying_unknown_reals = targets,
    target_normalizer=None,#MultiNormalizer([TorchNormalizer() for _ in targets]),
    allow_missing_timesteps=True
)

validation = TimeSeriesDataSet.from_dataset(training,data,predict=True,stop_randomization=True)

batch_size = 32

train_dataloader = training.to_dataloader(train=True,batch_size=batch_size,num_workers=0)
val_dataloader = validation.to_dataloader(train=False,batch_size=batch_size*10,num_workers=0)

In [200]:
from pytorch_forecasting import Baseline

actuals = torch.cat([y for x, (y,weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
# 假設你希望將 `baseline_predictions` 移動到相同的設備
baseline_predictions = baseline_predictions.to(actuals.device)

# 現在兩者在相同設備上，可以進行計算
result = (actuals - baseline_predictions).abs().mean().item()

/home/undergrad/miniconda3/envs/114/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/home/undergrad/miniconda3/envs/114/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/undergrad/miniconda3/envs/114/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


In [202]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TemporalFusionTransformer

early_stop_callback = EarlyStopping(monitor="val_loss",min_delta=1e-4,patience=5,verbose=True,mode="min")
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")

trainer = pl.Trainer(
    max_epochs=45,
    accelerator = 'gpu',
    devices=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[lr_logger,early_stop_callback],
    logger=logger
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.001,
    hidden_size=160,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=160,
    output_size=7,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4
)

trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

ValueError: Expected a parent